## Fonction creation NetCDF 3D CHL CMEMS

#### load packages

In [4]:
import collections
#import datetime
from datetime import datetime
from datetime import timedelta
import numpy
import xarray

#### Fonction to create the NetCDF from vectors depth, time, longitude and latitude 
#### variable 4D has to have (len(time), len(depth), len(latitude), len(longitude)) dimension

In [5]:
date = datetime.now()
def creation_NetCDF_3D_PRODUCT_CMEMS_N(path, date_today, date_product, depth_input,latitude_input, longitude_input,
                                       variable_4d_input1, variable_4d_error_input1, variable_4d_input2, variable_4d_error_input2,
                                       variable_4d_input3, variable_4d_error_input3):
    date_ref = datetime.strptime('1950-01-01', '%Y-%m-%d')
    date_product_datetime = datetime.strptime(date_product,'%Y%m%d')
    month = date_product_datetime.month
    month = str(month).zfill(2)
    delta = date_product_datetime - date_ref
    hours = delta.total_seconds()/3600
    time_input = numpy.ndarray(shape=(1,),dtype="float")
    time_input[0,] = hours
    print(' '.join([str(hours),"hours between 1950-01-01 and date"]))
    time = numpy.empty((1, ), dtype="float")
    depth = numpy.empty((36, ), dtype="int16")
    latitude = numpy.empty((689, ))
    longitude = numpy.empty((1440, ))
    # creation of NetCDF dimensions: time, depth, latitude and longitude
    coords = collections.OrderedDict(
        time=xarray.DataArray(time_input,
                              dims=("time", ),
                              coords=dict(time=time),
                              attrs=collections.OrderedDict(axis="T",
                                                            long_name="Time (hours since 1950-01-01 00:00:00)",
                                                            standard_name="time",
                                                            calendar = "Gregorian",
                                                            units="hours since 1950-01-01 00:00:00")),

        depth=xarray.DataArray(depth_input,
                               dims=("depth", ),
                               coords=dict(depth=depth),
                               attrs=collections.OrderedDict(axis="Z",
                                                             long_name="Depth",
                                                             positive="down",
                                                             standard_name="depth",
                                                             unit_long="Meters",
                                                             units="m")),
        latitude=xarray.DataArray(latitude_input,
                                  dims=("latitude", ),
                                  coords=dict(latitude=latitude),
                                  attrs=collections.OrderedDict(axis="Y",
                                                                long_name="Latitude",
                                                                standard_name="latitude",
                                                                step=numpy.float32(0.25),
                                                                unit_long="Degrees North",
                                                                units="degrees_north")),
        longitude=xarray.DataArray(longitude_input,
                                   dims=("longitude", ),
                                   coords=dict(longitude=longitude),
                                   attrs=collections.OrderedDict(axis="X",
                                                                 long_name="Longitude",
                                                                 standard_name="longitude",
                                                                 step=numpy.float32(0.25),
                                                                 unit_long="Degrees East",
                                                                 units="degrees_east")))
    
    #Creation of the variable in 4 dimensions: here Chl and poc with associated errors
    data_vars = collections.OrderedDict(

        Micro=xarray.DataArray(variable_4d_input1,
                            dims=("time", "depth", "latitude", "longitude"),
                            attrs=collections.OrderedDict(long_name="Microphytoplankton concentraion",
                                                          standard_name="mass_concentration_of_microphytoplankton_in_sea_water",
                                                          unit_long="milligram of microphytoplankton per cubic meter",
                                                          units="mg m-3")),
        Micro_error=xarray.DataArray(variable_4d_error_input1,
                            dims=("time", "depth", "latitude", "longitude"),
                            attrs=collections.OrderedDict(long_name="Error on microphytoplankton concentration",
                                                          standard_name="mass_concentration_of_microphytoplankton_in_sea_water",
                                                          unit_long="milligram of microphytoplankton per cubic meter",
                                                          units="mg m-3",
                                                          comment="Contains the error on the estimated values of microphytoplankton")),
        
    
        Nano=xarray.DataArray(variable_4d_input2,
                            dims=("time", "depth", "latitude", "longitude"),
                            attrs=collections.OrderedDict(long_name="Nanophytoplankton concentraion",
                                                          standard_name="mass_concentration_of_nanophytoplankton_in_sea_water",
                                                          unit_long="milligram of nanophytoplankton per cubic meter",
                                                          units="mg m-3")),
        Nano_error=xarray.DataArray(variable_4d_error_input2,
                            dims=("time", "depth", "latitude", "longitude"),
                            attrs=collections.OrderedDict(long_name="Error on nanophytoplankton concentration",
                                                          standard_name="mass_concentration_of_nanophytoplankton_in_sea_water",
                                                          unit_long="milligram of nanophytoplankton per cubic meter",
                                                          units="mg m-3",
                                                          comment="Contains the error on the estimated values of nanophytoplankton")),
    

        
        Pico=xarray.DataArray(variable_4d_input3,
                            dims=("time", "depth", "latitude", "longitude"),
                            attrs=collections.OrderedDict(long_name="Picophytoplankton concentraion",
                                                          standard_name="mass_concentration_of_picophytoplankton_in_sea_water",
                                                          unit_long="milligram of picophytoplankton per cubic meter",
                                                          units="mg m-3")),
        Pico_error=xarray.DataArray(variable_4d_error_input3,
                            dims=("time", "depth", "latitude", "longitude"),
                            attrs=collections.OrderedDict(long_name="Error on picophytoplankton concentration",
                                                          standard_name="mass_concentration_of_picophytoplankton_in_sea_water",
                                                          unit_long="milligram of picophytoplankton per cubic meter",
                                                          units="mg m-3",
                                                          comment="Contains the error on the estimated values of picophytoplankton")))    
    

    
    #Creation of the Dataset with dimensions (coords) and variable (data_vars) anthe genra attributes of the NetCDF file
    dataset = xarray.Dataset(
        data_vars=data_vars,
        coords=coords,
        attrs=collections.OrderedDict(title="Global Ocean monthly mean 3D Phytoplankton Functional Type (PFT) concentrations",
                                      project="BLUE CLOUD",
                                      project_url="https://blue-cloud.d4science.org/",
#                                       contact="servicedesk.cmems@mercator-ocean.eu",
#                                       description="CMEMS MULTIOBS 3D BGC monthly climatological global product estimated from merged satellite and hydrological data",
                                      Conventions="CF-1.6",
                                      institution="LOV/IMEV/CNRS",
                                      domain_name="GLO",
                                      references="Sauzede et al. 2015 (Journal of Geophysical Research: Oceans;  https://doi.org/10.1002/2014JC010355)"))

    dataset.to_netcdf(''.join([path,"/SOCA_PFT_glo_bgc3d_rep_2018_",month,"_P",str(date_today),".nc"]),
                      encoding=dict(time=dict(dtype=numpy.float32, _FillValue=None, zlib= True, complevel= 4),
                                    depth=dict(dtype=numpy.int16, _FillValue=None, zlib= True, complevel= 4),
                                    latitude=dict(dtype=numpy.float32, _FillValue=None, zlib= True, complevel= 4),
                                    longitude=dict(dtype=numpy.float32, _FillValue=None, zlib= True, complevel= 4),
                                    Micro=dict(dtype=numpy.float32, _FillValue=99999, zlib= True, complevel= 4),
                                    Micro_error=dict(dtype=numpy.float32, _FillValue=99999, zlib= True, complevel= 4),
                                    Nano=dict(dtype=numpy.float32, _FillValue=99999, zlib= True, complevel= 4),
                                    Nano_error=dict(dtype=numpy.float32, _FillValue=99999, zlib= True, complevel= 4),
                                    Pico=dict(dtype=numpy.float32, _FillValue=99999, zlib= True, complevel= 4),
                                    Pico_error=dict(dtype=numpy.float32, _FillValue=99999, zlib= True, complevel= 4)), unlimited_dims=["time"], mode="w")